# Trabajo Práctico 2: Capa de Transporte
## Redes y Transmisión de Datos, UNPSJB, Sede Trelew, 2018
---
## Cátedra:
   ### * Mg. Ricardo López
   ### * Lic. Nahuel Defossé
---
### Alumno:
   ### * Luciano Serruya Aloisi

# PARTE 1: TCP Fundamentos

## 1. Dos aplicaciones en las máquinas A y B intercambian datos utilizando el protocolo TCP.

### a) Durante el intercambio de datos entre A y B, luego de establecida la conexión, la máquina A no recibe una confirmación de B sobre su ultimo envío, por lo que retransmite el segmento no confirmado. Al cabo de un tiempo, B recibe las dos copias del segmento tal como A las envió. ¿B detecta que se trata de dos copias de la misma información? Justifique su respuesta.

El host B detecta que los segmentos recibidos son idénticos debido a que tiene el mismo *número de secuencia (SEQ)*; se queda con una copia y desecha el otro (*drop*). 

### b) Suponga que la conexión TCP que A inició con B, es una sesión telnet, (o sea que A se conectará al puerto 23 de B). ¿Es posible que A inicie una segunda conexión TCP hacia el puerto 23 de B, manteniendo la primera conexión activa?  Justifique su respuesta e indique, en caso de ser posible, cómo diferenciará A y B los segmentos correspondientes a cada una de las conexiones.

El proceso del host B que esté corriendo telnet, tendrá ocupado el puerto 23; el recurso lo tiene reservado pura y exclusivamente para él. Si el host A se quiere conectar por segunda vez al puerto 23 del host B, no tedrá respuesta ya que estará ocupado con la primer conexión.

Si el host A se logró conectar con el host B, entonces otro puerto estará siendo utilizado por el host B para manetener la conexión.

Los segmentos correspondientes a cada aplicación se identificarán unívocamente con una tupla formada de 4 valores:
* IP del emisor
* Puerto del emisor por el cuál están saliendo los datos 
* IP del destinatario
* Puerto del destinatario que está escuchando la conexión


## 2. Responda: Verdadero o Falso y justifique.
## El uso de ventanas de un protocolo:

### a) Permite aumentar la eficiencia de la transmisión.

Verdadero. Se pueden enviar varios paquetes secuencialmente, sin esperar el reconocimiento por parte del receptor (como sucede con los procotocolos de parada y espera). Cabe aclarar también que en protocolos tipo Go-Back-N, el uso de vetanas deslizantes puede jugar en contra, ya que se envían varios paquetes en simultáneo, pero si el receptor pide una retransmisión, se reenvían esos varios paquetes (generando más carga en la red).

### b) Garantiza el control de errores.

Falso. El protocolo de la capa de transporte (ya sea UDP o TCP) es el encargado de detectar los errores, y no el uso de las ventanas

### c) Requiere la identificación de las tramas.

Verdadero. Se identifican las tramas con un número de secuencia para saber:
* Cuál fue enviado y reconocido
* Cuál fue enviado y no reconocido todavía
* Cuál no fue enviado todavía
* Cuál no fue considerado todavía (están por fuera de la ventana)

### d) Disminuye el overhead del protocolo.

Falso. Justamente al contrario, incorporan un mayor trabajo al protocolo debido a que tienen que actualizar la ventana 

### e) Realiza el control de CRC.

Falso. El Control de Redundancia Cíclica le corresponde a la capa de enlace

### f) No acepta paquetes fuera de orden.

Falso. El protocolo Selective Repeat recibe paquetes fuera de orden y los almacena hasta que llegan los paquetes anteriores (con número de secuencia inferiores). Una vez recibidos todos los paquetes, se los entrega a la capa superior (de aplicación)

## 3. Utilice el software Cliente – Servidor de la parte 2 del TP1 y lance el analizador de protocolos para observar la siguiente secuencia:

### a) Establezca conexión entre el cliente y el servidor, enviando un campo de datos del orden de 10 caracteres. Cierre luego la conexión.

Corriendo en una terminal el servidor TCP de ECO con el comando `./bin/echo` y en otra ejecutar el cliente con el comando `echo "0123456789" | ./bin/tcp_client --echo`, se puede obtener con `tcpdump` una captura como la de `pcap/ejercicio3.pcap` 

### b) Indique la cantidad de paquetes circularon entre ambos extremos desde el inicio de la conexión hasta su cierre.

Para toda la comunicación circularon 10 paquetes
* 3 Para el 3-Way handshake
* 4 para el envío de los datos
* 3 más para el cierre de la conexión

### c) Observe e indique las banderas que se observan en el 3-Way y 4-Way de inicio y cierre de la conexión.

* Para el 3-Way handshake el cliente envía la bandera SYN, a lo que el servidor le contesta SYN y ACK. El cliente le envía ACK
* Para el 4-Way handshake (haciendo la captura en un host con una distribución de Linux), se observan 3 paquetes. El primero es del cliente al servidor con las banderas FIN y ACK; el servidor le responde FIN y ACK. Por último el servidor le envía un ACK.

### d) Indique el número de secuencia que se observa en ambos extremos al inicio de la conexión. Y la ventana de Flujo de ambos extremos.

#### SEQ -> Número de secuencia
#### WIN -> Tamaño de ventana

* Primer mensaje del cliente -> SEQ: 1608898858, WIN: 43690 
* Respuesta del servidor -> SEQ: 2170056674, WIN: 43690

### e) Indique los números de secuencia que se observan para ambos extremos al término del 3-Way state (sin envío de datos). Qué le parece particular?

Terminado el 3-Way handshake, los números de secuencia son secuenciales a partir del 1

### f) Indique los números de secuencia que se observan para ambos extremos al inicio del cierre de la conexión.

* Primer mensaje de fin de conexión del cliente -> SEQ: 19
* Respuesta del servidor -> SEQ: 35

### g) Indique los números de secuencia que se observan para ambos extremos al término del cierre de la conexión. Qué le parece particular?

Los números de secuencia de ambos hosts se incrementaron en 1 por más de no haber enviado datos durante la etapa de fin de la conexión.

## 4. Dado el siguiente segmento TCP (se excluye la cabecera IP), enviado por un host A a otro B:

```
04 02 00 50 00 00 32 98
00 00 00 00 60 02 20 00
13 03 00 00 02 04 05 B4
```

04 02 -> puerto fuente

00 50 -> puerto destino

00 00 32 98 -> SEQ

00 00 00 00 ACK


6    0    0    2
0110 xxxx xx00 0010

WIN 6

URG 0 
ACK 0
PSH 0
RST 0
SYN 1
FIN 0

20 00 WIN

13 03 CHECKSUM
00 00 URGENT DATA POINTER

02 04 05 B4 OPTIONS

### a) Qué función cumple el segmento enviado?

El segmento enviado es el primero de un 3-Way handshake para establecer la conexión con el host B ya que solamente tiene la bandera SYN levantada

### b) Caracterice a los Hosts A y B como cliente/servidor.

El Host A es el cliente ya que le está enviando al Host B (servidor) un segmento que solamente tiene levantada la bandera SYN

### d) Cuál será el número de secuencia que utilizará el Host A en el próximo segmento que le envíe a B?

El número de secuencia del próximo segmento que le envíe A a B será un 1

### e) Cuál es el servicio requerido?

El servicio requerido es el de un servidor web (el Host B es un servidor web), ya que la conexión es hacia el puerto 80 del Host B.

## 5. a. Suponga que la ventana de congestión de TCP está en 18 Kbytes. La ventana publicada por el otro extremo es de 64 Kbyte. ¿A qué valor llegará dicha ventana si los siguientes 5 segmentos transmitidos resultan exitosos y no se recibió aún ningún ACK? Suponga un tamaño máximo de segmento de 2 Kbytes.

Debido a que no llega ningún ACK por parte del receptor, el emisor no modifica su ventana de congestión.


Ahora bien, si esta situación perdura (no llega ningún ACK), eventualmente se vencerán los timers del receptor (*timeout*). Por lo tanto se el emisor actualizará el ```sstresh``` con el valor del ```cwnd``` / 2 = 9KBytes, y el ```cwnd``` a| 1 MSS (2 KBytes).

## b. Suponga ahora que luego de transmitir los 5 segmentos exitosos llega un ACK acumulativo reconociendo tres paquetes y que el umbral se encuentra en 12Kbytes.  A qué valor llegará la ventana de congestión.

Suponiendo que la variable `cwnd` está en 18 KBytes y el `sstresh` en 12KBytes, se puede asumir que el emisor está en modo *Congestion avoidance*. Por lo tanto, con el reconocimiento de 3 segmentos actualizará su ventana de congestión en 1 MSS por cada segmento reconocido, esto es, de 18 KBytes se incrementará a 24 KBytes (18 KBytes + 3 MSS)

## 6. Determine el tamaño óptimo de ventana para una sesión TCP en la que el RTT=100mseg, MSS=600 bytes y velocidad de la interfase 128 Kbps.

#### Bandwidth-in-bits-per-second * Round-trip-latency-in-seconds = TCP window size in bits / 8 = TCP window size in bytes
#### [Finding the optimal TCPIP receive window size](http://smallvoid.com/article/tcpip-rwin-size.html)

In [19]:
rtt = 100
bandwidth = 128
mss = 600

rwin = (bandwidth/8)*rtt

print('''El tamaño de ventana óptimo para una sesión TCP con:
    RTT = {rtt} ms
    MSS = {mss} bytes
    velocidad de la interfaz = {bandwidth} kbps
    
es igual a {rwin} bytes'''.format(
    rtt=rtt,
    mss=mss,
    bandwidth=bandwidth,
    rwin=int(rwin)))

El tamaño de ventana óptimo para una sesión TCP con:
    RTT = 100 ms
    MSS = 600 bytes
    velocidad de la interfaz = 128 kbps
    
es igual a 1600 bytes


## 7. Qué mejora aporta el mecanismo de retransmisión rápida (fast retransmit) de TCP? Indique si aporta mejora, si la empeora o si es indiferente. Justifique.

### A. Permite que no llegue a ocurrir congestión en la red.

Indiferente. El *fast retransmit* y la congestión se detectan bajo el mismo evento (3 ACKs seguidos para un mismo número de secuencia).

### B. Permite que un proceso detecte que hay congestión antes de que esta alcance un valor demasiado alto.

Mejora. El receptor está pidiendo retransmisión de un segmento que no le llegó (pudo haber sido desechado por congestión)

### C. Permite que la ventana de recepción pueda ser más grande.

Es indiferente. La retransmisión rápida no afecta los tamaños de ventana

### D. Aumenta la Eficiencia de la comunicación.

Mejora. Se reenvía un segmento antes de que se venza el timer, agilizando la comunicación.

### E. Permite que la congestión ocurra antes.

Empeora la situación. Circula una mayor cantidad de paquetes en la red

## 8. Supongamos tener una situación en la cual se observa que al realizar una transferencia de archivo por FTP entre dos nodos, se observa que la transferencia efectiva de los datos es excesivamente lenta para el ancho de banda disponible. O sea que el ancho de banda consumido por la transferencia del archivo es muy pequeño si se compara con el ancho de banda total disponible entre los dos nodos.  

## En cada uno de los casos siguientes, indique si la solución propuesta serviría para mejorar la situación y porqué (cómo afecta positiva o negativamente a la mejora de la velocidad de transferencia, o cómo no tiene ninguna influencia). Si los efectos de la solución dependen de otros parámetros no especificados, indiquese cuáles son estos.

### a. Se observa que la latencia de la red (retardo de transmisión) entre los dos procesos es muy alta. Como solución se propone aumentar el tamaño de la ventana de TCP en ambos nodos.

Aumentar el tamaño de ventana podría afectar negativamente ya que la alta latencia puede provenir de una congestión en la red; al agrandar la ventana, se enviarían más segmentos a la red, generando aún más congestión en la red.

Sin embargo, puede ser que el ancho de banda se esté subutilizando porque las ventanas de ambos nodos no alcancen para llevar la cantidad de segmentos enviada; por lo tanto, agrandar las ventanas podría ser beneficioso.

### b. Se observa que se están perdiendo muchos paquetes debido a errores en la red.  Sin embargo, la latencia es muy baja. Como solución se propone disminuir el timeout de espera por asentimientos en los dos nodos

Esta solución podría funcionar, porque al no tener latencia en la red, los paquetes que lleguen a destino van llegar rápido. Como los paquetes se están perdiendo por motivos desconocidos, esperar menos tiempo antes de retransmitirlo sería una buena opción

### c. Se observa que se están reenviando muchos paquetes innecesariamente, porque los paquetes “originales” han llegado a destino. Como solución se propone aumentar el tamaño de la ventana de TCP en ambos nodos.

No solucionaría el problema, ya que el problema reside en que el emisor no espera el tiempo suficiente para que el paquete llegue a destino y le llegue su ACK correspondiente

### d. Se observa que durante largos períodos cesa la emisión por llenarse la ventana de TCP del emisor. Como solución se propone disminuir el tamaño de la ventana del emisor y aumentar el plazo de espera por asentimientos


Disminuir el tamaño de ventana agravaría aún más la situación. La ventana del emisor se llena porque hay muchos segmentos los no se recibió su ACK. Aumentar el tamaño de ventana podría mejorar la situación.

Con respecto al timer, aumentar el plazo de espera retrasaría más la comunicación. 

# PARTE 2: Análisis de segmentos

### A)

![Diagrama de secuencia 9A](diagramas/A.svg)

---

### B)

![Diagrama de secuencia 9B](diagramas/B.svg)

---

### C)

![Diagrama de secuencia 9C](diagramas/C.svg)

---

### D)

![Diagrama de secuencia 9D](diagramas/D.svg)

---

### E.1)

![Diagrama de secuencia 9E.1](diagramas/E1.svg)

---

### E.2)

![Diagrama de secuencia 9E.2](diagramas/E2.svg)

# PARTE 3: TCP Análisis de Tramas

## 10. Análisis de capturas de tráfico de red e interpretación del mismo. Ahora vamos a estudiar e interpretar el segmento TCP que se encuentra dentro del paquete IP capturado.

#### Cabecera IP
```
4    5    0    0       0    1    9    5
0100 0101 0000 0000    0000 0001 1001 0101
1    2    A    C       4    0    0    0
0001 0010 1010 1100    0100 0000 0000 0000
8    0    0    6       6    3    4    1
1000 0000 0000 0110    0110 0011 0100 0001
C    0    A    8       0    1    2    2
1100 0000 1010 1000    0000 0001 0010 0010
C    0    A    8       0    1    0    3
1100 0000 1010 1000    0000 0001 0000 0011

```

* Version: 0x4 (IPv4)
* IHL: 0x5
* Type of service: 0x0
* Total length: 0x195
* Identification: 0x12AC
* Flags: 0x4 (X)
* Fragment Offset: 0x0
* TTL: 0x80
* Protocol: 0x6 (TCP)
* Header Checksum: 0x6341
* SOURCE ADDRESS: 0xC0A80122 (192.168.1.34)
* DESTINATION ADDRESS: 0xC0A80103 (192.168.1.3)
 
--- 

#### Segmento TCP
```
0    5    C    0      0    C    E    A
0000 0101 1100 0000   0000 1100 1110 1010
2    7    D    D      4    4    A    3
0010 0111 1101 1101   0100 0100 1010 0011
6    F    A    D      2    5    3    B
0110 1111 1010 1101   0010 0101 0011 1011
5    0    1    8      F    D    5    9
0101 0000 0001 1000   1111 1101 0101 1001
2    D    E    F      0    0    0    0
0010 1101 1110 1111   0000 0000 0000 0000
6    9    0    1      0    0    0    0
0110 1001 0000 00001  0000 0000 0000 0000
```

* SOURCE PORT: 0x05C0 (1472)
* DESTNATION PORT: 0x0CEA (3306)
* SEQ: 0x27DD44A3
* ACK: 0x6FAD253B
* Data offset: 0x5
* Reserved: 0x0
* Flags:
    * URG: 0
    * ACK: 1
    * PSH: 1
    * RST: 0
    * SYN: 0
    * FIN: 0
* WIN: 0xFD59
* Checksum: 0x2DEF
* Urgent Pointer: 0x0
* Options: 0x690100
* Padding: 0x0

#### La sección de datos comienza en el word de 16 bits `0x0355`

## 11.Se realizó la captura de las siguientes tramas Ethernet. (tenga en cuenta que se extrajeron los bytes de preámbulo). Se pide: Analizar los campos relevantes de la información de nivel de transporte que contienen

### 1) 
```
00 18 f8 4e 70 2f 00 50 2c a4 34 ec 08 00 45 00
00 3e 7f 5e 00 00 80 11 cf aa c0 a8 01 64 c8 2a
61 6f 04 06 00 35 00 2a 2c a8 e4 e8 01 00 00 01
00 00 00 00 00 00 03 77 77 77 08 6d 69 6e 69 6e
6f 76 61 03 6f 72 67 00 00 01 00 01 23 cd ac f2
```

#### IP
* Protocol: 0x11 (UDP)
* Source Address: 0xC0A80164 (192.168.1.100)
* Destination Address: 0xC82A616F (200.34.97.111)

#### Transporte (UDP)

* Source port: 0x0406 (1030)
* Desintation port: 0x35 (53, servicio DNS)
* Length: 0x2A
* Checksum: 0x2CA8

Se le está enviando a un servidor de DNS la dirección "www.mininiva.org"

### 2)
```
00 50 2c a4 34 ec 00 18 f8 4e 70 2f 08 00 45 00
00 5c b7 fa 00 00 3c 11 da f0 c8 2a 61 6f c0 a8 01
64 00 35 04 06 00 48 36 15 e4 e8 81 80 00 01 00
02 00 00 00 00 03 77 77 77 08 6d 69 6e 69 6e 6f
76 61 03 6f 72 67 00 00 01 00 01 c0 0c 00 05 00
01 00 00 25 11 00 02 c0 10 c0 10 00 01 00 01 00
00 25 11 00 04 57 e9 93 8c a1 23 64 f3
```

#### IP
* Protocol: 0x11 (UDP)
* Source Address: 0xC82A616F (200.34.97.111)
* Destination Address: 0xC0A80164 (192.168.1.100)


#### Transporte (UDP)

* Source port: 0x35 (53, serivcio DNS)
* Desintation port: 0x406 (1030)
* Length: 0x48
* Checksum: 0x3615

Respuesta del servidor DNS por la petición del inciso anterior

### 3)
```
00 18 f8 4e 70 2f 00 50 2c a4 34 ec 08 00 45 00
00 30 7f 61 40 00 80 06 cd e4 c0 a8 01 64 57 e9
93 8c 0c 54 00 50 f0 e8 a3 97 00 00 00 00 70 02
ff ff 34 79 00 00 02 04 05 b4 01 01 04 02 11 ac fb
4b
```

#### IP
* Protocol: 0x6 (TCP)
* Source Address: 0xC0A80164 (192.168.1.100)
* Desintation Address: 0x57E9938C (87.233.147.140)


#### Transporte (TCP)

* Source port: 0xC54 (3156)
* Desintation port: 0x50 (80, servidor web)
* SEQ: 0xF0E8A397
* ACK: 0x0
* Data offset: 0x7
* Flags:
    * URG: 0
    * ACK: 0
    * PSH: 0
    * RST: 0
    * SYN: 1
    * FIN: 0
* WIN: 0xFFFF
* Checksum: 0x3479
* Urgent Pointer: 0x0
* Options: 0x020405
* Padding: 0xB4

La bandera de `SYN` está levantada, es la primer etapa del *3-Way handshake* para establece una conexión con el host `87.233.157.140` (que es un servidor web)

### 4)
```
00 50 2c a4 34 ec 00 18 f8 4e 70 2f 08 00 45 00
00 30 00 00 40 00 35 06 98 46 57 e9 93 8c c0 a8
01 64 00 50 0c 54 16 3b ae 0d f0 e8 a3 98 70 12
06 d0 59 4f 00 00 02 04 05 b4 01 01 04 02 ac 23
23 ca
```
#### IP
* Protocol: 0x6 (TCP)
* Source Address: 0x57E9938C (87.233.147.140)
* Desintination Address: 0xC0A80164 (192.168.1.100)



#### Transporte (TCP)

* Source port: 0x50 (80, servidor web)
* Destination port: 0xC54 (3156)
* SEQ: 0x163BAE0D
* ACK: 0xF0E8A398
* Data offset: 0x7
* Flags:
    * URG: 0
    * ACK: 1
    * PSH: 0
    * RST: 0
    * SYN: 1
    * FIN: 0
* WIN: 0x06D0
* Checksum: 0x594F
* Urgent Pointer: 0x0
* Options: 0x020405
* Padding: 0xB4

Segmento `SYN ACK` para el *3-Way handshake* del inciso anterior

### 5)
```
00 18 f8 4e 70 2f 00 50 2c a4 34 ec 08 00 45 00
00 28 7f 62 40 00 80 06 cd eb c0 a8 01 64 57 e9
93 8c 0c 54 00 50 f0 e8 a3 98 16 3b ae 0e 50 10
ff ff 9c e3 00 00 a2 cb 23 45
```

#### IP
* Protocol: 0x6 (TCP)
* Source Address: 0xC0A80164 (192.168.1.100)
* Destination Address: 0x57E9938C (87.233.147.140)




#### Transporte (TCP)

* Source port: 0xC54 (3156)
* Desination port: 0x50 (80, servidor web)
* SEQ: 0xF0E8A398
* ACK: 0x163BAE0E
* Data offset: 0x5
* Flags:
    * URG: 0
    * ACK: 1
    * PSH: 0
    * RST: 0
    * SYN: 0
    * FIN: 0
* WIN: 0xFFFF
* Checksum: 0x9CE3
* Urgent Pointer: 0x0
* Options: 0xA2CB23
* Padding: 0x45

Último segmento `ACK` del establecimiento de la conexión entre un cliente y un servidor web

### 6)
```
00 18 f8 4e 70 2f 00 50 2c a4 34 ec 08 00 45 00
01 c0 7f 63 40 00 80 06 cc 52 c0 a8 01 64 57 e9
93 8c 0c 54 00 50 f0 e8 a3 98 16 3b ae 0e 50 18
ff ff ee 95 00 00 47 45 54 20 2f 20 48 54 54 50 2f
31 2e 31 0d 0a 48 6f 73 74 3a 20 77 77 77 2e 6d
69 6e 69 6e 6f 76 61 2e 6f 72 67 0d 0a 55 73 65
72 2d 41 67 65 6e 74 3a 20 4d 6f 7a 69 6c 6c 61
2f 35 2e 30 20 28 57 69 6e 64 6f 77 73 3b 20 55
3b 20 57 69 6e 64 6f 77 73 20 4e 54 20 35 2e
……………………..
```

#### IP
* Protocol: 0x6 (TCP)
* Source Address: 0xC0A80164 (192.168.1.100)
* Destination Address: 0x57E9938C (87.233.147.140)


#### Transporte (TCP)

* Source port: 0xC54 (3156)
* Desination port: 0x50 (80, servidor web)
* SEQ: 0xF0E8A398
* ACK: 0x163BAE0E
* Data offset: 0x5
* Flags:
    * URG: 0
    * ACK: 1
    * PSH: 1
    * RST: 0
    * SYN: 0
    * FIN: 0
* WIN: 0xFFFF
* Checksum: 0xEE95
* Urgent Pointer: 0x0
* Options: 0x474554
* Padding: 0x20

Petición *HTTP* al sitio www.mininova.org (ruta /) realizada desde un navegador *Mozilla*, corriendo en un Windows